In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 17.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 90.7 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('BaseGATGraphModel')
experiment.add_tags(['movielens', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/1e74ff5a183140f2be3be0cc15587fff

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 3e-4,
    'train_batch_size': 4096,
    'train_print_every': 10,  
    'train_test_every': 10,
    'test_topk': 10,
    'test_batch_size': 2048
}

In [7]:
import os
os.listdir('/kaggle/input/data/leave-n-out/mvln')

['train.csv', 'test.csv']

In [8]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [9]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [10]:
rootpath = '/kaggle/input/data/leave-n-out/mvln/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['timestamp'], unit='s')
print(train.head())

   user_id  movie_id  rating  timestamp                date
0        1      3186       4  978300019 2000-12-31 22:00:19
1        1      1270       5  978300055 2000-12-31 22:00:55
2        1      1721       4  978300055 2000-12-31 22:00:55
3        1      1022       5  978300055 2000-12-31 22:00:55
4        1      2340       3  978300103 2000-12-31 22:01:43


In [11]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 211802
Количество explicit негативного фидбека 153484


In [12]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 327987
Количество implicit негативного фидбека 246536


In [13]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','movie_id','target','date']]
train.head()

,user_id,movie_id,target,date
0,1,3186,implicit_positive,2000-12-31 22:00:19
1,1,1270,explicit_positive,2000-12-31 22:00:55
2,1,1721,implicit_positive,2000-12-31 22:00:55
3,1,1022,explicit_positive,2000-12-31 22:00:55
4,1,2340,implicit_negative,2000-12-31 22:01:43


In [14]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)

In [15]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['timestamp'], unit='s')
print(test.head())

   user_id  movie_id  rating  timestamp                date
0        1      2687       3  978824268 2001-01-06 23:37:48
1        1       745       3  978824268 2001-01-06 23:37:48
2        1       588       4  978824268 2001-01-06 23:37:48
3        1         1       5  978824268 2001-01-06 23:37:48
4        1      2355       5  978824291 2001-01-06 23:38:11


In [16]:
test = test[['user_id','movie_id', 'date']]
test.head()

,user_id,movie_id,date
0,1,2687,2001-01-06 23:37:48
1,1,745,2001-01-06 23:37:48
2,1,588,2001-01-06 23:37:48
3,1,1,2001-01-06 23:37:48
4,1,2355,2001-01-06 23:38:11


# MVP model v2

In [17]:
test = test[(test.user_id.isin(train.user_id)) & (test.movie_id.isin(train.movie_id))].copy()
test.shape

(60394, 3)

In [18]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'movie_id'] = video_encoder.fit_transform(train['movie_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'movie_id'] = video_encoder.transform(test['movie_id'])

In [19]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['movie_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных movie_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных movie_id 3700
Количество уникальных user_id 6040


In [20]:
def build_full_graph(num_nodes: int) -> torch.Tensor:
    idx = torch.arange(num_nodes)
    src = idx.repeat(num_nodes)
    dst = idx.unsqueeze(1).repeat(1, num_nodes).view(-1)
    return torch.stack([src, dst], dim=0)


def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['movie_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'movie_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    # user <-> user
    # data['user', 'interacts', 'user'].edge_index = build_full_graph(num_users)
    return data

In [21]:
class HeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        for t, h in h1.items():
            h1[t] = self.dropout(F.leaky_relu(self.norm1[t](h)))

        h2 = self.conv2(h1, data.edge_index_dict)
        out = {}
        for t, h in h2.items():
            out[t] = self.norm2[t](h)
            
        return out['user']

In [22]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[6040] },
  item={ node_id=[3700] },
  implicit_positive={ node_id=[6040] },
  explicit_positive={ node_id=[6040] },
  implicit_negative={ node_id=[6040] },
  expliсit_negative={ node_id=[6040] },
  (item, to_feedback_implicit_positive, implicit_positive)={ edge_index=[2, 327987] },
  (implicit_positive, feedback_to_item_implicit_positive, item)={ edge_index=[2, 327987] },
  (implicit_positive, to_user_implicit_positive, user)={ edge_index=[2, 6040] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 211802] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 211802] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 6040] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 246536] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 246536] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 604

In [23]:
train.movie_id.nunique(), train.movie_id.min(), train.movie_id.max()

(3700, 0, 3699)

In [24]:
num_users = len(train['user_id'].unique())
num_items = train['movie_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = HeteroGNN(num_users, num_items, feedback_types)

In [25]:
model

HeteroGNN(
  (user_emb): Embedding(6040, 32)
  (item_emb): Embedding(3700, 32)
  (fb_emb): ModuleDict(
    (implicit_positive): Embedding(6040, 32)
    (explicit_positive): Embedding(6040, 32)
    (implicit_negative): Embedding(6040, 32)
    (expliсit_negative): Embedding(6040, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implic

In [26]:
model(data)

tensor([[-0.5632,  0.2609, -1.3624,  ..., -2.5372, -0.5059, -1.3198],
        [-0.4295, -1.5870,  0.0169,  ..., -1.5316, -1.1555, -1.0729],
        [-0.0791,  0.3499,  0.4089,  ...,  2.3449, -0.8924, -1.6724],
        ...,
        [-0.7345, -0.9524,  0.6445,  ..., -0.1650, -0.9734, -0.6582],
        [-1.4275, -0.9545, -0.2927,  ..., -0.6495, -0.3625, -1.6914],
        [ 0.1451, -1.0966, -0.1414,  ..., -0.7511, -1.5610, -1.8678]],
       grad_fn=<NativeLayerNormBackward0>)

In [27]:
test_df = test[['user_id', 'movie_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'movie_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["movie_id"].agg(set).to_dict()

In [28]:
def train_model(model: HeteroGNN,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048) -> HeteroGNN:
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 0
    print(f"Num of training examples: {num_train}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                model.eval()
                # model.to('cpu')
                with torch.no_grad():
                    user_emb = model(train_data)
                    item_emb = model.item_emb.weight
                    item_emb_t = item_emb.t().detach()
                    del item_emb
                    gc.collect()

                    scores = []
                    for i in range(0, user_emb.shape[0], test_batch_size):
                        user_e = user_emb[i:i+test_batch_size]
                        rating = torch.mm(user_e, item_emb_t)
                        _, topk_items = torch.topk(rating, k=test_top_k, dim=1)
                        scores.extend(topk_items)

                        del user_e, rating
                        gc.collect()

                    scores = torch.cat(scores, dim=0)
                    scores = scores.reshape((num_users, test_top_k))
                    topk_np = scores.cpu().numpy()

                    filtered_topk = []
                    for u_idx in range(num_users):
                        seen = viewed_items[u_idx]
                        recs = topk_np[u_idx, :]
                        mask = np.isin(recs, list(seen), invert=True)
                        filtered = recs[mask][:top_k] 
                        filtered_topk.append(filtered)
                    
                    users = []
                    items = []
                    ranks = []
                    for u_idx in range(num_users):
                        recs = filtered_topk[u_idx]
                        for rank, item in enumerate(recs[:top_k], 1):
                            users.append(u_idx)
                            items.append(item)
                            ranks.append(rank)
                    
                    reco_df = pd.DataFrame({
                        Columns.User: users,
                        Columns.Item: items,
                        Columns.Rank: ranks,
                    })

                    metrics = {
                        'map@10': MAP(k=10),
                        'precision@10': Precision(k=10),
                        'recall@10': Recall(k=10),
                        'ndcg@10': NDCG(k=10)
                    }
                    
                    results = calc_metrics(
                        metrics=metrics,
                        reco=reco_df,
                        interactions=interactions,
                    )
                print(f"Step/epoch {step}/{epoch}, Test metrics:")
                for key, value in results.items():
                    print(f"{key}: {value:.9f}")
                    experiment.log_metric(f"Test {key} vs step", value, step=total_steps)
                del scores, topk_np
                gc.collect()

                model.to(device)
                model.train()
                train_data.to(device)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [29]:
experiment.log_parameters(hyperparameters)

In [30]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [31]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 211802
Epoch 1, Step 1, LR: 0.000300, Current Loss: 3.1944, Avg Loss: 3.1944
Diff stats — min: -28.7889, max: 28.8653, mean: 0.0818, std: 7.8812

Step/epoch 1/1, Test metrics:
precision@10: 0.002864238
recall@10: 0.002866078
ndcg@10: 0.002995891
map@10: 0.000922238
Epoch 1, Step 10, LR: 0.000300, Current Loss: 3.2057, Avg Loss: 3.1835
Diff stats — min: -25.7316, max: 25.9495, mean: 0.0453, std: 7.8804

Step/epoch 10/1, Test metrics:
precision@10: 0.002913907
recall@10: 0.002915747
ndcg@10: 0.002922583
map@10: 0.000867070
Epoch 1, Step 20, LR: 0.000300, Current Loss: 3.0501, Avg Loss: 3.1449
Diff stats — min: -28.7978, max: 32.7993, mean: 0.1262, std: 7.6568

Step/epoch 20/1, Test metrics:
precision@10: 0.002814570
recall@10: 0.002816409
ndcg@10: 0.002892458
map@10: 0.000878962
Epoch 1, Step 30, LR: 0.000300, Current Loss: 3.0533, Avg Loss: 3.1121
Diff stats — min: -25.0935, max: 27.1920, mean: 0.1927, std: 7.7264

Step/epoch 30/1, Test metrics:
precision@10: 0

In [32]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [33]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [34]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [35]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : BaseGATGraphModel
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/1e74ff5a183140f2be3be0cc15587fff
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [600] : (0.04525217413902283, 3.480424642562866)
COMET INFO:     Diff stats (std) vs step [600]  : (2.2659318447113037, 7.881175518035889)
COMET INFO:     Test map@10 vs step [600]       : (0.000778349364028172, 0.012498083762570517)
COMET INFO:     Test ndcg@10 vs step [600]      : (0.0028448401385444367, 0.034820011766281876)
COMET INFO:     Test precision@10 vs step [600] : (0.002798013245033113, 0.03327814569536424)
COMET INFO:  